# COGS 108 - Data Checkpoint

# Names

- Hyunjo Lee
- Jaehoon Choi
- Maya Que
- Muhammad Imran
- Yiliang Yuan

<a id='research_question'></a>
# Research Question

Do economic factors (including GDP, oil price, inflation rate, unemployment, S&P 500) correlate with the number of overall seats the incumbent party wins or loses in an election? If so, which one of these factors influence the results the most?

# Dataset(s)

*Fill in your dataset information here*

(Copy this information for each dataset)
- Dataset Name:
- Link to the dataset:
- Number of observations:

1-2 sentences describing each dataset. 

If you plan to use multiple datasets, add 1-2 sentences about how you plan to combine these datasets.


1. GDP
- Dataset Name: Real Gross Domestic Product(GDP)
- Link to the dataset: https://fred.stlouisfed.org/series/GDPC1 
- Number of observations: 301
- Descipton: This dataset contains the real GDP from 1947 Q1 to 2022 Q1. We can derive the real GDP growth rate from this dataset.

2. Inflation Rate
- Dataset Name: 300 years of inflation rate in US
- Link to the dataset: https://www.kaggle.com/datasets/prasertk/300-years-of-inflation-rate-in-us 
- Number of observations: 321
- Descipton: This dataset contains annual inflation rate from 1700-2022 in the United States.

3. S&P 500
- Dataset Name: S&P 500 total value and percentage change data
- Link to the dataset: https://data.nasdaq.com/data/MULTPL/SP500_REAL_PRICE_MONTH-sp-500-real-price-by-month\n
- Number of observations: 1838
- Descipton: This dataset contains the total value and percentage change in S&P 500 index from 1871.

4. Unemployment
- Dataset Name: Unemployment Rate in US
- Link to the dataset: https://data.bls.gov/timeseries/LNS14000000?years_option=all_yearss
- Number of observations: 891
- Descipton: This dataset contains the unemployment rate in US from 1948.

5. Crude Oil
- Dataset Name: Crude Oil Price in US
- Link to the dataset: https://fred.stlouisfed.org/series/WTISPLC
- Number of observations: 915
- Descipton: This dataset contains the price of the crude oil in US from 1946.

With all of the above economic datasets, we will clean each one separately, and keep monthly data from 1950 Janurary to 2022 March. For datasets which do not have month to month data, we will repeat the rows to fit the format(e.g. we will repeat the rows of yearly data for 12 times). At the end of the data cleaning process, we will combine all of the cleaned dataset to a single table.

6. Generic Ballot Poll
- Dataset Name: Generic Ballot Poll (1945~)
- Link to the dataset: https://assets.datacamp.com/production/repositories/1934/datasets/b67f87da8083695e8d49387a37d0ff30f341bded/generic_ballot.csv
- Number of observations: 2559
- Descipton: This dataset contains results of biweekly polling asking "which party's candidate would you vote for? Republican or Democrat?"

7. Party divisions of United States Congresses
- Dataset Name: Party divisions by Congress
- Link to the dataset: https://en.wikipedia.org/wiki/Party_divisions_of_United_States_Congresses#References
- Number of observations: 36
- Descipton: This dataset contains, for each congress, the number of senators and house representatives by party as well as the incumbent president.

With all of the above political datasets, we will clean each one separately, and keep data from 1950 Janurary to 2022 March.


# Setup

In [1]:
import pandas as pd
import numpy as np
from functools import reduce        

# Data Cleaning

Describe your data cleaning steps here.

In [2]:
## YOUR CODE HERE
## FEEL FREE TO ADD MULTIPLE CELLS PER SECTION

# https://www.kaggle.com/datasets/prasertk/300-years-of-inflation-rate-in-us 
inflation_rate = pd.read_csv("./data/inflation_rate_yearly.csv")
inflation_rate = inflation_rate.iloc[249:]
inflation_rate = inflation_rate.drop(columns='amount')

# https://fred.stlouisfed.org/series/WTISPLC
crude_oil = pd.read_csv("./data/crude_oil.csv")
crude_oil = crude_oil.iloc[48:]
crude_oil = crude_oil.rename(columns={"WTISPLC": "Crude Oil"}).reset_index(drop=True)

# https://data.bls.gov/timeseries/LNS14000000?years_option=all_yearss
unemployment_rate = pd.read_csv("./data/unemployment_rate.csv")
unemployment_rate = unemployment_rate.iloc[2:]

# https://fred.stlouisfed.org/series/GDPC1 
gdp = pd.read_csv("./data/GDPC1.csv")
gdp = gdp.iloc[11:]
gdp = gdp.rename(columns={"GDPC1": "GDP"})

# https://data.nasdaq.com/data/MULTPL/SP500_REAL_PRICE_MONTH-sp-500-real-price-by-month\n"
snp500_percent = pd.read_csv('./data/SNP500_%change.csv')
snp500_percent = snp500_percent.rename(columns={"Value": "S&P500 Percent Change"})

snp500_total = pd.read_csv('./data/SP500_REAL_PRICE_MONTH.csv')
snp500_total = snp500_total.rename(columns={"Value": "S&P500 Total Value"})

Date = crude_oil['DATE']

The unemployment rate table we are using right now is not in the form we want to use. For our purpose, we should combine the year and month to a single column called 'Date', and put the corresponding unemployment rate on its right side. The table we expect to get should only contain two columns.

In [3]:
unemployment_array =  unemployment_rate.iloc[:,1:]
unemployment_array = unemployment_array.values.tolist()
unemployment_array = np.concatenate(unemployment_array, axis=None)[:-9]
unemployment_data = {'Date':Date, 'Unemployment Rate':unemployment_array}
unemployment = pd.DataFrame(data = unemployment_data).reset_index(drop=True)
unemployment = unemployment.rename(columns={'Date':'DATE'})

The real GDP data we have right now only represent absolute values from which we are unable to understand the economic enviornment in a certain period of time. Therefore, we decide to use the real GDP growth rate which can reflect the general health of the economy in our research. From Bureau of Economic Analysis https://www.bea.gov/help/faq/463, we can get the formula for calculating the real gdp growth rate. With this formula, we will get the growth rate of our real GDP data.

In [4]:
GDP_growth = []
total_terms = len(gdp['GDP'])
GDP_Q = gdp['GDP'].tolist()
for i in range(0,total_terms-1):
    # Get the annulized ratio in GDP between each consecutive months 
    annual_r = (GDP_Q[i+1]/GDP_Q[i])**4
    # Get the growth rate of real GDP
    growth_r = (annual_r-1)*100
    GDP_growth.append(growth_r)

gdp = gdp.iloc[1:,:]
gdp['GDP growth'] = GDP_growth
gdp = gdp.reset_index(drop = True)

For the S&P 500 index, we want to combine the datasets of its total value and its percentage change to a single table.

In [5]:
snp500 = pd.merge(snp500_percent, snp500_total)
snp500 = snp500.sort_values(by='Date').reset_index(drop=True)
snp500_index = snp500.index[snp500['Date']=='1950-01-01'].tolist()
snp500 = snp500.iloc[947:]
snp500 = snp500.rename(columns={'Date':'DATE'})
snp500 = snp500.reset_index(drop=True)

Repeat the rows of datasets which do not have monthly data.

In [6]:
rep_inflation = pd.DataFrame(np.repeat(inflation_rate.values, 12, axis=0), columns=inflation_rate.columns).iloc[:-9]
rep_inflation = rep_inflation.rename(columns={'year':'DATE'})
rep_inflation = rep_inflation.rename(columns={'inflation rate':'Inflation Rate'})
rep_inflation['DATE'] = Date
rep_gdp = pd.DataFrame(np.repeat(gdp.values, 3, axis=0), columns=gdp.columns)
rep_gdp['DATE'] = Date

Now, we are trying to combine all the tables together. 

In [17]:
result = [crude_oil, unemployment, rep_inflation, rep_gdp, snp500]
result_merged = reduce(lambda  left,right: pd.merge(left,right,on=['DATE'],how='outer'), result)
result_merged = result_merged[:867]
display(result_merged)

,DATE,Crude Oil,Unemployment Rate,Inflation Rate,GDP,GDP growth,S&P500 Percent Change,S&P500 Total Value
0,1950-01-01,2.57,6.5,0.01,2186.365,16.671652,0.020556,16.88
1,1950-02-01,2.57,6.4,0.01,2186.365,16.671652,0.019550,17.21
2,1950-03-01,2.57,6.3,0.01,2186.365,16.671652,0.008135,17.35
3,1950-04-01,2.57,5.8,0.01,2253.045,12.768758,0.028242,17.84
4,1950-05-01,2.57,5.5,0.01,2253.045,12.768758,0.033632,18.44
...,...,...,...,...,...,...,...,...
862,2021-11-01,79.15,4.2,0.05,19806.29,6.894521,0.046333,4667.39
863,2021-12-01,71.71,3.9,0.05,19806.29,6.894521,0.023598,4674.77
864,2022-01-01,83.22,4.0,0.03,19735.895,-1.414108,-0.040359,4573.82
865,2022-02-01,91.64,3.8,0.03,19735.895,-1.414108,-0.017621,4435.98


In [8]:
# https://assets.datacamp.com/production/repositories/1934/datasets/b67f87da8083695e8d49387a37d0ff30f341bded/generic_ballot.csv
generic_ballot = pd.read_csv('./data/generic_ballot.csv')
generic_ballot['Date'] = pd.to_datetime(generic_ballot['Date'])
generic_ballot = generic_ballot.sort_values(by='Date').reset_index(drop=True)
index = generic_ballot.index[generic_ballot['Date'] >= '1950-01-01'].tolist()
generic_ballot = generic_ballot.iloc[27:].reset_index()
display(generic_ballot)

,index,Date,Democrats,Republicans,ElecYear,ElecDay,DaysTilED,DemVote,RepVote
0,27,1950-01-13,41,35,1950,11/7/1950,298,49.6,48.9
1,28,1950-02-01,39,30,1950,11/7/1950,279,49.6,48.9
2,29,1950-02-10,45,23,1950,11/7/1950,270,49.6,48.9
3,30,1950-05-09,47,31,1950,11/7/1950,182,49.6,48.9
4,31,1950-05-09,54,36,1950,11/7/1950,182,49.6,48.9
...,...,...,...,...,...,...,...,...,...
2527,2554,2016-11-05,45,46,2016,11/8/2016,3,48.0,49.1
2528,2555,2016-11-06,45,46,2016,11/8/2016,2,48.0,49.1
2529,2556,2016-11-06,48,43,2016,11/8/2016,2,48.0,49.1
2530,2557,2016-11-06,45,48,2016,11/8/2016,2,48.0,49.1


In [9]:
# https://en.wikipedia.org/wiki/Party_divisions_of_United_States_Congresses#References
govt_party_div = pd.read_csv('./data/govt_party_division.csv')
govt_party_div = govt_party_div.drop(columns='Congress')
govt_party_div[['Start','End']] = govt_party_div['Years'].str.split('–', expand=True)
govt_party_div = govt_party_div.drop(columns='Years')
govt_party_div.columns = ['Senate_Total','Senate_Dem','Senate_Rep','Senate_Others','Senate_Vacancies','House_Total','House_Dem','House_Rep','House_Others','House_Vacancies','President','Start','End']
display(govt_party_div)

,Senate_Total,Senate_Dem,Senate_Rep,Senate_Others,Senate_Vacancies,House_Total,House_Dem,House_Rep,House_Others,House_Vacancies,President,Start,End
0,96,54,42,—,—,435,262,171,2,—,Harry S. Truman,1949,1951
1,96,48,47,1,—,435,235,199,1,—,Harry S. Truman,1951,1953
2,96,46,48,2,—,435,213,221,1,—,Dwight D. Eisenhower,1953,1955
3,96,48,47,1,—,435,232,203,—,—,Dwight D. Eisenhower,1955,1957
4,96,49,47,—,—,435,234,201,—,—,Dwight D. Eisenhower,1957,1959
5,98,64,34,—,—,437,284,153,—,—,Dwight D. Eisenhower,1959,1961
6,100,64,36,—,—,437,262,175,—,—,John F. Kennedy,1961,1963
7,100,67,33,—,—,435,258,176,—,1,Lyndon B. Johnson,1963,1965
8,100,68,32,—,—,435,295,140,—,—,Lyndon B. Johnson,1965,1967
9,100,64,36,—,—,435,247,187,—,1,Lyndon B. Johnson,1967,1969


The datasets are now combined into one table and organized neatly by the measurements we are observing and by date; thus, it is clean. Some of the data were already in a usable format because it was organized by date in chronological order and had the correct values we wanted to measure. For the other data, we had to drop some unnecessary columns and calculate the rates using the given values. We also checked the data distributions to make sure that the data was reasonable and had appropriate patterns. The cleaned data can be used to begin the data analysis and determine the results of our research question. 